In [ ]:
import pandas as pd
import numpy as np
import scipy.signal
import seaborn as sns
import matplotlib.ticker as ticker
import itertools
import math

In [ ]:
path = "/home/francisco/tps/datos/tp2/"
load_path = path + "{}_transformado.hdf"

N_P = 1
N_S = 1
pacientes_P = []
pacientes_S = []
for load_name, N, dest in [("P", N_P, pacientes_P), ("S", N_S, pacientes_S)]:
    for paciente in [load_name + "{:02d}".format(i) for i in range(1, 1 + N)]:
        dest.append(pd.read_hdf(load_path.format(paciente)))

In [3]:
def calcular_delta(d,e):
    mean_1_inicial = d[e == e[0]].mean()
    mean_2_inicial = d[e != e[0]].mean()
    return mean_2_inicial - mean_1_inicial


def permutation_test(d,e, n=100):
    d0 = calcular_delta(d, e)
    deltas = np.zeros(n)
    aux_e = e.copy()

    for i in range(n):
        np.random.shuffle(aux_e)
        deltas[i] = calcular_delta(d,aux_e)

    return deltas, d0

def foo(a, e_a, b, e_b):
    e_1 = np.full(a.shape[0], e_a) 
    e_2 = np.full(b.shape[0], e_b)
    e = np.concatenate((e_1, e_2)) 
    d = np.concatenate((a,b))
    return d,e

def test_permutacion(soleado, lluvioso):
    d, e = foo(soleado, 's', lluvioso, 'l')

    # delta-value : la diferencia entre las medias que presenta nuestra información 
    # p-value es la probabilidad de haber obtenido este delta o mayor

    deltas, delta_value = permutation_test(d,e)
    p_value =  deltas[deltas >= delta_value].shape[0]/deltas.shape[0]

    return p_value

def entropia(df_):
    return -(df_["probabilidad"] * np.log(df_["probabilidad"]))
    

In [ ]:
entropias_P = []
entropias_S = []
for my_entropy_list, dfs_ in [(entropias_P, pacientes_P), (entropias_S, pacientes_S)]:
    for p in dfs_:
        entropias = p.groupby(["epoch", "sensor"]).reset_index(name="entropia")
        my_entropy_list.append(entropias["entropia"].mean())
        
df_entropias = pd.DataFrame({
    "Capacidad cognitiva": (["Reducida"] * N_P) + (["Normal"] * N_S),
    "Entropia": entropias_P + entropias_S
})

ymin = min(df_b['Entropia'])
ymax = max(df_b['Entropia'])
decimo = (ymax - ymin)/len(df_b['Entropia'])
ymin, ymax = ymin - decimo, ymax + decimo

sns.barplot(x="Capacidad cognitiva", y="Entropia", data=df_entropias, palette="Set2")
ax = sns.swarmplot(x="Capacidad cognitiva", y="Entropia", data=df_entropias, palette="Set2")
ax.set_ylim([ymin, ymax])
sns.plt.show()


p_values = {}            
p_values[banda] = test_permutacion(df_entropias[df_entropias['Capacidad cognitiva'] == "Reducida"]['Entropia'], 
                                   df_entropias[df_entropias['Capacidad cognitiva'] == "Normal"]['Entropia'])
print("Probabilidad de que tengan misma media {}".format(banda, p_values[banda]))
